# Homework 3: DataFrames, Control Flow, and Probability

## Due Saturday, October 28th at 11:59PM

Welcome to Homework 3! This homework will cover lots of different topics:
- Grouping with subgroups (see [BPD 11](https://notes.dsc10.com/02-data_sets/groupby.html#subgroups))
- Merging DataFrames (see [BPD 13](https://notes.dsc10.com/02-data_sets/merging.html))
- Conditional statements (see [CIT 9.1](https://inferentialthinking.com/chapters/09/1/Conditional_Statements.html))
- Iteration (see [CIT 9.2](https://inferentialthinking.com/chapters/09/2/Iteration.html))
- Probability (see [CIT 9.5](https://inferentialthinking.com/chapters/09/5/Finding_Probabilities.html))

### Instructions

Remember to start early and submit often. You are given six slip days throughout the quarter to extend deadlines. See the syllabus for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

**Important**: For homeworks, the `otter` tests don't usually tell you that your answer is correct. More often, they help catch careless mistakes. It's up to you to ensure that your answer is correct. If you're not sure, ask someone (not for the answer, but for some guidance about your approach). These are great questions for office hours (the schedule can be found [here](https://dsc10.com/calendar)) or Ed. Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. 

In [ ]:
# Please don't change this cell, but do make sure to run it.
import babypandas as bpd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (10, 5)

import numpy as np
import otter
grader = otter.Notebook()

# We need to import some extra packages for some fun demonstrations.
import json
from ipywidgets import interact, widgets
from IPython.display import YouTubeVideo, HTML, display, clear_output, Image, IFrame

# Don't worry about this.
def read_json(path):
    f = open(path, 'r')
    return json.load(f)

### Supplemental Video on DataHub and Jupyter Notebooks

In Lab 0, we linked you to a video that walks you through key ideas you should be aware of when working on DataHub and in Jupyter Notebooks, including
- how files are organized on DataHub
- what it means to "restart the kernel"
- how to use keyboard shortcuts (most important: use `SHIFT + ENTER` to run a cell!)

We're linking it here again for your convenience. If you feel a little shaky on how to work your way around a notebook or troubleshoot issues, we recommend you give it another watch. (When troubleshooting, make sure to always check the [Debugging](https://dsc10.com/debugging/) tab on the course website as well.)

The video is quite long, but if you open the video directly on YouTube (which you can do by clicking the video's title after it loads in the next cell) you'll see timestamps in the description which you can use to jump to different parts of the video depending on what you'd like to learn more about.

In [ ]:
# Run this cell.
YouTubeVideo('Hq8VaNirDRQ')

## 1. 100 Years of "K" Baby Names 👶🏻

As we saw in [Lecture 8](https://dsc10.com/resources/lectures/lec08/lec08.html#Example:-Common-first-names), 7 out of the 525 students in DSC 10 this quarter are named Kevin! In this problem, we'll look more generally at baby names starting with the letter "K". The file `data/baby_names.csv` contains information from the [Social Security Administration](https://www.ssa.gov/oact/babynames/limits.html) about "K" baby names in the US from the past 100 years. Run the cell below to read in the data.

In [ ]:
baby = bpd.read_csv('data/baby_names.csv')
baby

The DataFrame `baby` has a row for each `'State'` (50 US states plus Washington DC), `'Gender'` (`'M'` or `'F'`, as assigned at birth), `'Year'` (between 1923 and 2021), and `'Name'`. The `'Count'` column records the number of babies of that gender who were given that name in one state in one year.

Below, we look at only the rows corresponding to the name `'Kevin'`.

In [ ]:
baby[baby.get('Name') == 'Kevin']

The first row of the DataFrame shows that there were 5 male babies named Kevin born in Connecticut in 1923. There are many other rows corresponding to a `'Name'` of Kevin, which come from other years, other states, and also female babies named Kevin, of which there are some!


Run the cell below to find out when and where many female Kevins were born.

In [ ]:
female_kevins = baby[(baby.get('Name') == 'Kevin') & (baby.get('Gender') == 'F')]
female_kevins.sort_values(by='Count', ascending=False)

**Question 1.1.** There are many more male Kevins than female Kevins. Create a line plot that shows how the number of male babies named Kevin has changed over time in the US. Then use your plot to answer the question that follows.

In [ ]:
# Create your line plot here.

Around what year was the peak in popularity for the name Kevin for male babies? Choose the closest answer from the options below and set `male_kevin_peak` to 1, 2, 3, or 4 corresponding to your answer choice.
1. 1960
1. 1970
1. 1980
1. 1990

In [ ]:
male_kevin_peak = ...

In [ ]:
grader.check("q1_1")

**Question 1.2.** In the `baby` DataFrame, how many babies of each gender were born in each state? Create a DataFrame named `num_babies` with one row for each gender in each state and columns `'State'`, `'Gender'`, and `'Count'`, which contains the total number of babies of each gender in each state with a "K" name. The first few rows of `num_babies` are shown below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>State</th>
      <th>Gender</th>
      <th>Count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>AK</td>
      <td>F</td>
      <td>14892</td>
    </tr>
    <tr>
      <th>1</th>
      <td>AK</td>
      <td>M</td>
      <td>8217</td>
    </tr>
    <tr>
      <th>2</th>
      <td>AL</td>
      <td>F</td>
      <td>118774</td>
    </tr>
    <tr>
      <th>3</th>
      <td>AL</td>
      <td>M</td>
      <td>72058</td>
    </tr>
  </tbody>
</table>

***Hints:***
- You can do this in one line of code.
- Don't forget to use `.reset_index()`.


In [ ]:
num_babies = ...
num_babies

In [ ]:
grader.check("q1_2")

A gendered name is a combination of a name and a gender, such as female Kevins. Let's explore the average age of people with each gendered name. For example, let's calculate the average age of all female Kevins.

In [ ]:
female_kevins

We'll define the age of a person as 2023 (the current year) minus the year in which the person was born. This doesn't take into account people's birthdays, because we don't have that information. For example, if a female Kevin was born in 1983, they will be counted as 2023 - 1983 = 40 years old. Therefore the **total age** of all the female Kevins is given below.

In [ ]:
total_age = ((2023 - female_kevins.get('Year')) * female_kevins.get('Count')).sum()
total_age

To find the average age, we need to know how many female Kevins there are. The **total count** of female Kevins is given below.

In [ ]:
total_count = female_kevins.get('Count').sum()
total_count

Therefore the **average age** of female Kevins is given below.

In [ ]:
average_age = total_age / total_count
average_age

Notice that we _cannot_ calculate the average age of female Kevins as follows.

In [ ]:
age = 2023 - female_kevins.get('Year')
age.mean()

This is incorrect because it does not take into account the fact that there were more female Kevins born some years than others.

**Question 1.3.** Create a DataFrame named `avg_age` that has one row for each gendered name and columns `'Gender'`, `'Name'`, and `'Average_Age'`, which contains the average age of all people with each gendered name. The first few rows of `avg_age` are shown below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Gender</th>
      <th>Name</th>
      <th>Average_Age</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>F</td>
      <td>Ka</td>
      <td>33.372781</td>
    </tr>
    <tr>
      <th>1</th>
      <td>M</td>
      <td>Ka</td>
      <td>32.000000</td>
    </tr>
    <tr>
      <th>2</th>
      <td>F</td>
      <td>Kacey</td>
      <td>26.783084</td>
    </tr>
    <tr>
      <th>3</th>
      <td>M</td>
      <td>Kacey</td>
      <td>15.402124</td>
    </tr>
  </tbody>
</table>

***Hints:***
- Before attempting this question, make sure you understand the strategy shown above for finding the average age of female Kevins. You will need to generalize this approach.
- This is a multi-step problem. Add cells and display your intermediate results so you can see your progress as you go.
- You should check that the average age for female Kevins in your DataFrame `avg_age` is the same as we found above.


In [ ]:
avg_age = ...
avg_age

In [ ]:
grader.check("q1_3")

## 2. Settling a Debate 🥊

Some of the DSC 10 staff members are arguing about who has the best taste in movies. They decide to settle the debate by seeing whose favorite movie was most successful at the box office, since better movies typically earn more money. 

Run the cell below to see the favorite movies of the staff members who are participating in this "contest" and store this data in a DataFrame named `staff_favorite`.

In [ ]:
staff_favorite = bpd.read_csv('data/favorite_movies.csv')
staff_favorite

The `staff_favorite` DataFrame does not contain the information we need to settle the debate, namely, the box office earnings of the movie. For this information, we can refer to data collected from [Box Office Mojo](https://www.boxofficemojo.com/). Run the cell below to load in a dataset containing the top 1000 highest-grossing box office movies and save it as a DataFrame named `top_1000`.

In [ ]:
top_1000 = bpd.read_csv('data/box_office_top_1000.csv')
top_1000

**Question 2.1.** Using the `merge` method, combine the `staff_favorite` and `top_1000` DataFrames, and assign the resulting DataFrame to the variable `combined`. `combined` should contain all of the columns in both `staff_favorite` and `top_1000`, minus the `'Favorite'` column from `staff_favorite`, which is redundant with the `'Movie'` column from `top_1000`.

In [ ]:
combined = ...
combined

In [ ]:
grader.check("q2_1")

**Question 2.2.** If you completed Question 2.1 correctly, you'll notice that `combined` has fewer rows than both `staff_favorite` and `top_1000`. This is because there are some movies that are in `staff_favorite` and not in `top_1000`, and other movies that are in `top_1000` and not in `staff_favorite`. 

Below, assign `staff_favorite_not_top_1000` to the **number** of different movies that are in `staff_favorite` and not in `top_1000`. Similarly, assign `top_1000_not_staff_favorite` to the number of different movies that are in `top_1000` and not in `staff_favorite`.

***Hint:*** There are two ways to find the number of unique values in a column.

1. Group by that column. On the resulting DataFrame, use `.shape[0]`.

2. Use the `.unique()` method on the Series corresponding to that column. Use `len` on the resulting array.

You'll need to do this three times – once for each of the columns containing movie names in `staff_favorite`, `top_1000`, and `combined`.


<!--
BEGIN QUESTION
name: q2_2
-->

In [ ]:
staff_favorite_not_top_1000 = ...
top_1000_not_staff_favorite = ...

print('There are', staff_favorite_not_top_1000, 'movies in `staff_favorite` that are not in `top_1000`.')
print('There are', top_1000_not_staff_favorite, 'movies in `top_1000` that are not in `staff_favorite`.')

In [ ]:
grader.check("q2_2")

**Question 2.3.** Now that we better understand how `combined` came to be, let's use it to find the staff member whose favorite movie earned the most money at the box office. Set the name of this person to `movie_connoisseur`. 

<!--
BEGIN QUESTION
name: q2_3
-->

In [ ]:
movie_connoisseur = ...
movie_connoisseur

In [ ]:
grader.check("q2_3")

**Question 2.4.** Some staff members whose names do not appear in `combined` are complaining that their favorite movies were not even considered as contenders for the winner of most box office earnings. They think we need to redo the analysis with a larger dataset about box office earnings, so their favorite movies can be included as well, giving them a fair chance to win.

Do you think the method we used to find the staff member whose favorite movie had the highest box office earnings was fair? Set `fair_method` to a boolean value, `True` or `False`.
<!--
BEGIN QUESTION
name: q2_4
-->

In [ ]:
fair_method = ...
fair_method

In [ ]:
grader.check("q2_4")

## 3. Death and Taxes 💰💸

The United States, like many countries, uses a progressive tax bracket system. This means that as your earnings increase, the percentage of your earnings you owe in tax also increases. In addition, the US tax system uses marginal tax brackets – what this means is that US taxpayers pay different tax percentages on different "chunks" of their earnings.

This year, taxes were due on April 18th, however on March 22, 2023, most California counties (including San Diego) received a blanket 6 month "disaster" extension, making the new deadline October 16th. Recently on October 16, 2023, the deadline was once again pushed back to November 16th, giving taxpayers an additional month to file their taxes. If you want to read more about this extension, here's a link to a [news article](https://www.nbcsandiego.com/nbc-7-responds-2/the-tax-deadline-was-extended-again-for-most-californians-heres-now-when-your-taxes-are-due/3329029/) about it. Let's suppose the tax brackets for the 2023 tax year for single filers are defined by the table below. These are pretty close to the actual brackets, but for simplicity's sake we'll use 5 brackets instead of 7. 

| Tax Bracket | Taxable Income |
| --- | --- |
| 10% | [$0, \\$11,000] |
| 12% | (\\$11,000, \\$44,725] |
| 22% | (\\$44,725, \\$95,375] |
| 24% | (\\$95,375, \\$182,100] |
| 32% | Over \\$182,100 |

**You will need to use these numbers throughout this question.**

A few notes:
- The notation $(a, b]$ means "greater than $a$ and less than or equal to $b$". For example, someone with a taxable income of \\$44,725 is in the 12% bracket, but someone with a taxable income of \\$44,725.01 is in the 22% bracket.
- "Taxable income" refers to the part of one's income that is taxable; in the US there is what's known as a "standard deduction" which can be thought of as a discount on your taxes. In this question, we won't worry about deductions.

If someone has a taxable income of \\$75,000, we say they are in the 22% tax bracket. However, such an individual doesn't owe 22% of \\$75,000 in taxes. Instead, they owe:
- 10% of \\$11,000, **plus**
- 12% of \\$33,725 (which is \\$44,725 - \\$11,000), **plus**
- 22% of \\$30,275 (which is \\$75,000 - \\$44,725).

More concretely, their tax owed is
$$0.1 \cdot \$11{,}000 + 0.12 \cdot \$33{,}725 + 0.22 \cdot \$30{,}275 = \$11{,}807.50.$$ 

This makes their **effective tax rate**
$$\frac{\$11{,}807.50}{\$75{,}000} = 0.15743,$$
or 15.743\%, because they wound up having to pay 15.743\% of their taxable income in taxes. We will give effective tax rates as percentages, rounded to three decimal places.

If you want to read more about the US federal income tax system, click [here](https://www.taxpolicycenter.org/briefing-book/how-do-federal-income-tax-rates-work).

**Question 3.1.** Complete the implementation of the function `tax_bracket`, which takes in a taxable income (`income`) and returns the tax bracket it is in, as a **proportion**. For instance, `tax_bracket(75000)` should evaluate to `0.22` and `tax_bracket(402150)` should evaluate to `0.32`.

For your convenience, the tax bracket table is shown again below.

| Tax Bracket | Taxable Income |
| --- | --- |
| 10% | [$0, \\$11,000] |
| 12% | (\\$11,000, \\$44,725] |
| 22% | (\\$44,725, \\$95,375] |
| 24% | (\\$95,375, \\$182,100] |
| 32% | Over \\$182,100 |

***Hints:*** 
- Use `elif`.
- To avoid typos and make your code more readable, you can use underscores in place of commas when inputting large numbers in Python. For example, you can write the number 1 million as `1000000` or `1_000_000`. 

In [ ]:
def tax_bracket(income):
    ...

# Feel free to change the line below to try other examples.
tax_bracket(75000) # should be 0.22

In [ ]:
grader.check("q3_1")

**Question 3.2.** Complete the implementation of the function `tax_owed`, which takes in a taxable income (`income`) and returns the amount of tax owed by an individual with that taxable income. For instance, `tax_owed(75000)` should evaluate to `11807.5` as in the example at the start of Question 3.

For your convenience, the tax bracket table is shown again below.

| Tax Bracket | Taxable Income |
| --- | --- |
| 10% | [$0, \\$11,000] |
| 12% | (\\$11,000, \\$44,725] |
| 22% | (\\$44,725, \\$95,375] |
| 24% | (\\$95,375, \\$182,100] |
| 32% | Over \\$182,100 |

***Hint:*** Below are some suggested steps to break down this problem into manageable chunks.
1. Make sure you understand the example calculating the tax owed on \\$75,000 of taxable income. 
2. Once you're sure you understand the given example calculation, then calculate the tax owed for a different taxable income, say \\$152,000, by hand. 
3. In a blank cell in your notebook, write Python code to compute the tax owed for \\$152,000 in taxable income and verify you get the same answer as when you did it by hand. 
4. Once you have that working, write the more general function and test it on a variety of inputs.

In [ ]:
def tax_owed(income):
    ...

# Feel free to change the line below to try other examples.
tax_owed(75000) # should be 11807.5

In [ ]:
grader.check("q3_2")

**Question 3.3.** Finally, complete the implementation of `effective_tax_rate`, which takes in a taxable income (`income`) and returns the effective tax rate for an individual with that taxable income, as a **percentage** rounded to three decimal places. For instance, `effective_tax_rate(75000)` should evaluate to `15.743` because someone who earns $75,000 of taxable income winds up having to pay 15.743\% of that amount in taxes. 

If `income` is less than or equal to 0, `effective_tax_rate` should evaluate to `0`.

***Hint:*** You should use your `tax_owed` function. Our entire solution is only four lines, and two of them are for handling the case when `income <= 0`.

In [ ]:
def effective_tax_rate(income):
    ...


# Feel free to change the line below to try other examples.
effective_tax_rate(75000) # should be 15.743

In [ ]:
grader.check("q3_3")

### Fun Demo

You just did all of the math necessary to build an income tax calculator, like the ones you see on many financial websites. Run the following cell once you've completed the rest of this question!

In [ ]:
# Don't worry about the code, just play with the slider that appears after running.
def plot_income_calculator(income):
    s = f'''
    <h1>DSC 10 Tax Calculator 💰💸</h1>
    <h3>{'${:,.2f}'.format(income)}</h3>
    Taxable Income
    
    <h2>{'${:,.2f}'.format(tax_owed(income))}</h2>
    Tax Owed
    
    <h3>{'{:,.1f}'.format((effective_tax_rate(income))) + '%'}</h3>
    Effective Tax Rate
    
    '''
    display(HTML(s))
    
interact(plot_income_calculator, income=(0, 200000, 1000));

## 4. Triton Tweets 🧜‍♂️💬


<img src = "data/triton_logo.jpg" width=200>

UCSD is starting it's own messaging app called Triton Tweets exclusive to staff and students! One of the key features of Triton Tweets is that when students send out messages, an emoji representing their college is automatically added to the end of their message. Your job in this question is to decide what emoji represents each college and to add these emojis to their tweets, using Python dictionaries and `for`-loops.

 We'll start by reminding you about Python dictionaries, which we also looked at briefly in Homework 2 (in the question about diamonds 💎). Dictionaries provide a convenient way to store data in `key: value` pairs. Here's an example dictionary you also saw in Homework 2.

In [ ]:
dsc_courses = {
    # key: value
    'DSC 10': 'Principles of Data Science',
    'DSC 20': 'Programming and Basic Data Structures for Data Science',
    'DSC 30': 'Data Structures and Algorithms for Data Science',
    'DSC 40A': 'Theoretical Foundations of Data Science I',
    'DSC 40B': 'Theoretical Foundations of Data Science II',
    'DSC 80': 'The Practice and Application of Data Science'
}

We can access the value corresponding to each key using bracket notation.

In [ ]:
dsc30_name = dsc_courses['DSC 30']
dsc30_name

Here, `'DSC 30'` is the key and `'Data Structures and Algorithms for Data Science'` is the value.

We can extract all keys from a dictionary using the .keys() method:

In [ ]:
dsc_courses.keys()

We can iterate through all of the keys (or values) of a dictionary if needed. For example:

In [ ]:
for course in dsc_courses.keys():
    print('I am taking', course)

In Python, emojis can be included as part of a string. For example:

In [ ]:
'🐍'

If you remove the quotes from the emoji above and run the cell, you will see `SyntaxError: invalid character`. Make sure that throughout this question, your emojis are contained within strings! (Fun fact, they cannot currently be used as variable names. Try it and see what error you get!)

The website [getemoji.com](https://getemoji.com/) allows you to copy and paste emojis. To select an emoji, double click it to highlight it. Try copying and pasting an emoji of your choice from that site into the code cell below. Alternatively, you can launch an emoji keyboard on your computer; if you're on a Mac, hit the `Control`, `Command`, and `Space` keys on your keyboard at the same time, and if you're using Windows, hit the `Windows` and `.` keys at the same time.

Don't forget to enclose your chosen emoji in quotes!

In [ ]:
# Copy and paste any emoji into this code cell.

**Question 4.1** In the cell below, create a dictionary called `colleges` that has the following eight keys:
- `'<Revelle>'` 
- `'<Muir>'`
- `'<Marshall>'`
- `'<Warren>'`
- `'<ERC>'`
- `'<Sixth>'`
- `'<Seventh>'`
- `'<Eighth>'`

The reason why we are writing the college is <> brackets is because the username format for Triton Tweets is "@name\<college\>." Every username is made up of an @ symbol, the student's name, and their college in <> brackets. You can assume that <> characters will never exist within the name portion of the username. By the end of this question, \<college\> will be changed into an emoji.

The values corresponding to each of these eight keys must be emojis, but they can be any emoji you want, as long as:

- they are copied from the site [getemoji.com](https://getemoji.com/), and 
- they are not in the "New Emojis" category at the bottom.

Have fun with it! We've chosen an emoji for `'<Revelle>'` as an example for you, but feel free to change it.

Some troubleshooting tips:

- After defining your dictionary, you may see some emoijs displayed with `'\U001...'` instead of their actual graphic. **If this happens, pick different emojis.**
- If you fail the test case that says your emojis are invalid, and you're certain you correctly defined your dictionary, you may consider choosing other emojis that are more generic that are more likely to be recognized by our autograder. This most likely won't be a problem if you copy your emojis from [getemoji.com](https://getemoji.com/) (the Mac and Windows emoji pickers use lots of new emojis, so it's possible you picked one of those).

In [ ]:
colleges = {
    '<Revelle>': '🌞',
    ...
}
colleges

In [ ]:
grader.check("q4_1")

Great! Now, if we want to create strings that use the college emojis, we don't need to copy-paste the emojis themselves – we can just access them from the `colleges` dictionary.

In [ ]:
colleges['<Sixth>']

In [ ]:
# Remember, the + symbol concatenates strings!
'@Ciro' + colleges['<ERC>']

Recall, in [Lecture 10](https://dsc10.com/resources/lectures/lec10/lec10.html#The-accumulator-pattern), we introduced the accumulator pattern. In the coin-flipping example, we started with an empty array, and added to it in each iteration of our `for`-loop.

We can also use the accumulator pattern for strings, by starting with an empty string, and repeatedly adding onto it via concatenation. Here's an example that loops through the letters in a word and replaces certain letters with emojis.

In [ ]:
output = ''
for ch in 'goodbye':
    if ch != 'o':
        output = output + ch
    else:
        output = output + colleges["<Revelle>"]
output

In the above example, we started with an empty string, `'output'`. For each character of the string `'goodbye'`, we added a single new character to `'output'`. If the character happened to be an 'o', it was replaced by the `'<ERC>'` emoji inside the `'colleges'` dictionary. 

**Question 4.2.** Now, complete the implementation of the function `emojify`, which take as input a string that represents a student's username, with an optional message. The username has the format "@name\<college\>." Every username will be made up of an @ symbol, the student's name, and their college in <> brackets. You can assume that <> characters will never exist within the name portion of the username.  The input can also include a message that goes after the username. 

The function should return a new string with the \<college\> replaced by the corresponding emoji from the `colleges` dictionary. Here is how the function would work on various example usernames and tweets. Your emojis may look different depending on what emojis you used in the `colleges` dictionary.

```py
>>> emojify('@Ylesia<ERC>: I love DSC10!')
'@Ylesia🌎: I love DSC10!'

>>> emojify('@Ester<Sixth>: Makai is the best dining hall!')
'@Ester🐈: Makai is the best dining hall!'

>>> emojify('@Mert<Warren>: I have not slept in days')
'@Mert🧸: I have not slept in days'
```

***Note:*** As we did in the preceding example, use the dictionary `colleges` to access the emojis – don't actually write any emojis within your function.

***Hints:*** 
- You will need to use a `for`-loop for this question. Refer to the example dictionary from the start of the problem (where we printed `'I am taking DSC 10'`) for some clues on how to start. Our solution code only adds two lines to the skeleton above!
- Use the string method [`.replace`](https://www.w3schools.com/python/ref_string_replace.asp). 

In [ ]:
def emojify(tweet):
    ...
        
    # Don't change this.
    return tweet

# An example call to emojify. Try out some other usernames too.
emojify('@Ciro<ERC>: Make sure to start the midterm project early!')

In [ ]:
grader.check("q4_2")

### Fun Demo

Run the cell below to produce a text box (don't worry about the code itself). Type a username (@name\<college\>) in the text box and watch it get emojified live! 

In [ ]:
def emojify_live(type_here):
    display(HTML('<h2>' + emojify(type_here) + '</h2>'))
interact(emojify_live, type_here="");

## 5. Alternating Means 🔁

In this problem, we'll define two functions that compute some sort of "alternating mean" of a sequence of values.

**Question 5.1.** Complete the implementation of the function `alternating_mean`, which takes in an array of numbers, `values`, and computes the mean of **every other element** in the original array starting with the element at position 0.

```py
>>> alternating_mean(np.array([3, 1.5, 4, 3.5]))
3.5 # Comes from (3 + 4) / 2.

>>> alternating_mean(np.array([3, 1.5, 4, 3.5, 5.9]))
4.3 # Comes from (3 + 4 + 5.9) / 3.

```

Once you've written your function, you should test it out on several arrays yourself to make sure it works as intended.

***Hint:*** Compute the mean by finding the sum of every other element in the array and dividing it by the number of elements that were summed.

<!--
BEGIN QUESTION
name: q5_1
-->

In [ ]:
def alternating_mean(values):
    ...
    
# Feel free to change this input to make sure your function works correctly.
alternating_mean(np.array([3, 1.5, 4, 3.5]))

In [ ]:
grader.check("q5_1")

**Question 5.2.** In math, the word "alternating" is sometimes used to indicate that the signs of the elements of a sequence oscillate, or "bounce", back and forth between positive and negative. Complete the implementation of the function `alternating_sign_mean`, which takes in an array of positive numbers, `values`, and computes the mean of **_every_ element** in the original array with **alternating signs**, starting with a positive sign for the element at position 0, a negative sign for the element at position 1, and so on. Example behavior is shown below.

```py
>>> alternating_sign_mean(np.array([3, 2, 9.5]))
3.5 # Comes from (3 + (-2) + 9.5) / 3.

>>> alternating_sign_mean(np.array([3, 2.5, 7, 6.5]))
0.25 # Comes from (3 + (-2.5) + 7 + (-6.5)) / 4.
```

***Hint:*** If `x` is an integer, `x % 2` evaluates to 0 when `x` is even and to 1 when `x` is odd. If `x` represents the position of an element in the array, you can use this to help you figure out whether the sign should be positive or negative.

<!--
BEGIN QUESTION
name: q5_2
-->

In [ ]:
def alternating_sign_mean(values):
    ...
    
# Feel free to change this input to make sure your function works correctly.
alternating_sign_mean(np.array([3, 2, 9.5]))

In [ ]:
grader.check("q5_2")

## 6. Lucky Triton Lotto 🔱 🎱 

Suppose UCSD holds an annual lottery called the Lucky Triton Lotto, where students can enter to win Triton Cash, or even free housing! Here's how the Lucky Triton Lotto works:

- First, you pick five **different** numbers, one at a time, from 1 to 28, representing that according to [USNews](https://www.usnews.com/best-colleges/university-of-california-san-diego-1317#:~:text=based%20academic%20calendar.-,University%20of%20California%2C%20San%20Diego's%20ranking%20in%20the%202022%2D2023,Jolla%20community%20of%20San%20Diego.), UCSD is ranked 28 in the nation for best universities to attend for 2022-2023.
- Then, you separately pick a number from 1 to 11. This is because UCSD's Data Science program is rank 11th in [USNews's](https://www.usnews.com/best-colleges/rankings/computer-science/data-analytics-science) Best Undergraduate Data Science Programs list. Let's say you select 3.
- The six numbers you have selected, or  **your numbers**, can be represented all together as (7, 12, 24, 15, 13, 3). This is a _sequence_ of six numbers – **order matters**!

The **winning numbers** are chosen by King Triton drawing five balls, one at a time, **without replacement**, from a pot of white balls numbered 1 to 28. Then, he draws a gold ball, the Tritonball, from a pot of gold balls numbered 1 to 11. Both pots are completely separate, hence the different ball colors. For example, maybe the winning numbers are (15, 9, 24, 23, 1, 3).

We’ll assume for this problem that in order to win the grand prize (free housing), all six of your numbers need to match the winning numbers and be in the **exact same positions**. In other words, your entire sequence of numbers must be exactly the same as the sequence of winning numbers. However, if some numbers in your sequence match up with the corresponding number in the winning sequence, you will still win some Triton Cash. 

Suppose again that you select (7, 12, 24, 15, 13, 3) and the winning numbers are (15, 9, 24, 23, 1, 3). In this case, two of your numbers are considered to match two of the winning numbers. 
- Your numbers: (7, 12, **24**, 15, 13, **3**)
- Winning numbers: (15, 9, **24**, 23, 1, **3**)

You won't win free housing, but you will win some Triton Cash. Note that although both sequences include the number 15 within the first five numbers (representing a white ball), since they are in different positions, that's not considered a match.


**Question 6.1.** What is the probability that your Tritonball number (the last number in your sequence) matches the winning Tritonball number? Calculate your answer and assign it to `tritonball_chance`. If you need to do any calculations (e.g. multiplication or division), make Python do it; don't use a separate calculator. Your result should be a decimal number between 0 and 1.

In [ ]:
tritonball_chance = ...
tritonball_chance

In [ ]:
grader.check("q6_1")

**Question 6.2.** What is the probability that your first three numbers match the first three winning numbers? Calculate your answer and assign it to `first_three_chance`. If you need to do any calculations (e.g. multiplication or division), make Python do it; don't use a separate calculator. Your result should be a decimal number between 0 and 1.

***Hint:*** You need **all three** of the first three numbers to match. What probability rule should you use?

In [ ]:
first_three_chance = ...
first_three_chance

In [ ]:
grader.check("q6_2")

**Question 6.3.** What is the probability that you win the grand prize, free housing? Calculate your answer and assign it to `free_housing_chance`. If you need to do any calculations (e.g. multiplication or division), make Python do it; don't use a separate calculator. Your result should be a decimal number between 0 and 1.

***Hint:*** When you select a ball without replacement, what happens to the total number of balls you can select next time?

In [ ]:
free_housing_chance = ...
free_housing_chance

In [ ]:
grader.check("q6_3")

**Question 6.4.** What is the probability that you do **not** win free housing? Calculate your answer and assign it to `no_free_housing_chance`. If you need to do any calculations (e.g. multiplication or division), make Python do it; don't use a separate calculator. Your result should be a decimal number between 0 and 1.

In [ ]:
no_free_housing_chance = ...
no_free_housing_chance

In [ ]:
grader.check("q6_4")

## Finish Line: Almost there, but make sure to follow the steps below to submit! 🏁

**_Citations:_** Did you use any generative artificial intelligence tools to assist you on this assignment? If so, please state, for each tool you used, the name of the tool (ex. ChatGPT) and the problem(s) in this assignment where you used the tool for help.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

Please cite tools here.

<hr style="color:Maroon;background-color:Maroon;border:0 none; height: 3px;">

To submit your assignment:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
1. Read through the notebook to make sure everything is fine and all tests passed.
1. Run the cell below to run all tests, and make sure that they all pass.
1. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope.
1. Stick around while the Gradescope autograder grades your work. Make sure you see that all tests have passed on Gradescope.
1. Check that you have a confirmation email from Gradescope and save it as proof of your submission.

In [ ]:
grader.check_all()